## Assignment #3

* Release date: 2021/04/26
* Due date: **2021/05/09 23:59** (will not accept late submission)
* Submittion format: notebook file which can be executed in Colab environment
* Weighting: 10% (total 100 pts)

* We will train CNN using `dogs_vs_cats_subset.zip` distributed in the class.

> ### (5pts) Prepare the dataset

* Place the unzipped files in some directory on your Colab instance.
* Count the number of JPEG files in `train`, `validation`, and `test` folders.

In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

# unzip
import zipfile, os, shutil

dataset = '/content/gdrive/My Drive/deeplearning/dogs_vs_cats_subset.zip'
dst_path = '/content/dogs_vs_cats_subset'
dst_file = os.path.join(dst_path, 'dogs_vs_cats_subset.zip')

if not os.path.exists(dst_path):
  os.makedirs(dst_path)

# copy zip file
shutil.copy(dataset, dst_file)
  
with zipfile.ZipFile(dst_file, 'r') as file:
  file.extractall(dst_path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
train_cats_dir = os.path.join(dst_path, 'subset/train/cats')
train_dogs_dir = os.path.join(dst_path, 'subset/train/dogs')

validation_cats_dir = os.path.join(dst_path, 'subset/validation/cats')
validation_dogs_dir = os.path.join(dst_path, 'subset/validation/dogs')

test_cats_dir = os.path.join(dst_path, 'subset/test/cats')
test_dogs_dir = os.path.join(dst_path, 'subset/test/dogs')

print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))

print('total validation cat images:', len(os.listdir(validation_cats_dir)))
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))

print('total test cat images:', len(os.listdir(test_cats_dir)))
print('total test dog images:', len(os.listdir(test_dogs_dir)))

total training cat images: 1000
total training dog images: 1000
total validation cat images: 500
total validation dog images: 500
total test cat images: 500
total test dog images: 500


> ### (20pts) Build a baseline model

* **(10pts)** Use VGG16 model as a baseline model.
  * You can use `tensorflow.keras.applications` module to get the VGG16 architecture.
  * You should customize the VGG16 model to deal with a given task, i.e., two class classification.
  * We will use 128*128 resized images as inputs to the model and randomly initialized model parameters.
  * Place **two output nodes** at the output layer.
  * Others not specified should be chosen yourself.

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import models
from tensorflow.keras import layers


conv_base = VGG16(include_top=False,
                  input_shape=(128, 128, 3))

conv_base.summary()

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

model.summary()

* **(10pts)** How many trainable parameters in the first convolutional layer? Explain how to compute the number of parameters.

첫번째 convolutinal layer를 보게 되면 학습가능한 파라미터의 개수가 1792개 인것을 알 수 있다. 이의 계산은 (input channal=3)X(output channel=64)X(filter size=3)X(filter size=3)+(output channel=64) = 1792 임을 알 수 있다.

> ### (10pts) Train a baseline model 

  * Currently, the data is stored as JPEG files. So we need the following steps:
    * Read the files.
    * Decode the JPEG content to RGB grids of pixels.
    * Convert these into floating-point tensors.
    * Scaling the data to be in a range of [0,1].
    
  * Set `batch_size` to 20.
  * Train the network for 50 epochs. It may consume some time. Note that you should set `steps_per_epoch` and `validation_steps` properly so that a particular data is processed once during a single epoch.
  * **Use Adam optimizer to train the model**. You may need to find hyperparameters (e.g., learning rate) to make the optimizer work.
  * Here, you don't need to apply any regularization methods. 

In [ ]:
from tensorflow.keras import optimizers

conv_base.trainable = False


model.compile(loss='CategoricalCrossentropy',
              optimizer=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999), 
              metrics=['acc'])



In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os 
train_dir = os.path.join(dst_path, 'subset/train')
validation_dir = os.path.join(dst_path, 'subset/validation')

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)



train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(128,128),
                                                    batch_size=20,
                                                    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(128,128),
                                                        batch_size=20,
                                                        class_mode='categorical')
history = model.fit_generator(train_generator,
                              steps_per_epoch=100,
                              epochs=50,
                              validation_data=validation_generator,
                              validation_steps=50)

> ### (5pts) Plot some curves
* Plot accuracies and losses on training and validation datasets, respectively.

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc'] 
val_acc = history.history['val_acc'] 
loss = history.history['loss'] 
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc') 
plt.plot(epochs, val_acc, 'b', label='Validation acc') 
plt.title('Training and validation accuracy') 
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss') 
plt.plot(epochs, val_loss, 'b', label='Validation loss') 
plt.title('Training and validation loss') 
plt.legend()

plt.show()

> ### (50pts) Improve the baseline model
* Try at least three strategies to improve the validation performance of the baseline model.
* You should examine that the performance is indeed improved as you employ your strategies. In other words, you should show that (accuracy with strategy 1 <= accuracy with strategy 1 and 2 <= accuracy with strategy 1, 2, and 3).

>> ### (15pts) Trial 1: something

input shape를 VGG16의 default값인 224,224로 바꾸어주고 dropout을 추가한다

In [ ]:

conv_base_2 = VGG16(include_top=False,
                  input_shape=(224, 224, 3))

model_2 = models.Sequential()
model_2.add(conv_base_2)
model_2.add(layers.Flatten())
model_2.add(layers.Dense(256, activation='relu'))
model_2.add(layers.Dropout(0.5))
model_2.add(layers.Dense(2, activation='softmax'))

conv_base_2.trainable = False


model_2.compile(loss='CategoricalCrossentropy',
              optimizer=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999), 
              metrics=['acc'])

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(224,224),
                                                    batch_size=20,
                                                    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(224,224),
                                                        batch_size=20,
                                                        class_mode='categorical')
history = model_2.fit_generator(train_generator,
                              steps_per_epoch=100,
                              epochs=50,
                              validation_data=validation_generator,
                              validation_steps=50)

acc = history.history['acc'] 
val_acc = history.history['val_acc'] 
loss = history.history['loss'] 
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc') 
plt.plot(epochs, val_acc, 'b', label='Validation acc') 
plt.title('Training and validation accuracy') 
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss') 
plt.plot(epochs, val_loss, 'b', label='Validation loss') 
plt.title('Training and validation loss') 
plt.legend()

plt.show()


>> ### (15pts) Trial 2: trial1 + something

train data에 여러가지 변화를 주어 train data의 유형과 수를 늘린다.

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(224,224),
                                                    batch_size=20,
                                                    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(224,224),
                                                        batch_size=20,
                                                        class_mode='categorical')

history = model_2.fit_generator(train_generator,
                              steps_per_epoch=100,
                              epochs=50,
                              validation_data=validation_generator,
                              validation_steps=50)



acc = history.history['acc'] 
val_acc = history.history['val_acc'] 
loss = history.history['loss'] 
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc') 
plt.plot(epochs, val_acc, 'b', label='Validation acc') 
plt.title('Training and validation accuracy') 
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss') 
plt.plot(epochs, val_loss, 'b', label='Validation loss') 
plt.title('Training and validation loss') 
plt.legend()

plt.show()



>> ### (20pts) Trial 3: trial1 + trial2 + something

learning rate를 더 작은 값으로 조정한다.

In [ ]:
model_2.compile(loss='CategoricalCrossentropy',
              optimizer=optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999),  #reduce learning rate
              metrics=['acc'])


history = model_2.fit_generator(train_generator,
                              steps_per_epoch=100,
                              epochs=50,
                              validation_data=validation_generator,
                              validation_steps=50)


acc = history.history['acc'] 
val_acc = history.history['val_acc'] 
loss = history.history['loss'] 
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc') 
plt.plot(epochs, val_acc, 'b', label='Validation acc') 
plt.title('Training and validation accuracy') 
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss') 
plt.plot(epochs, val_loss, 'b', label='Validation loss') 
plt.title('Training and validation loss') 
plt.legend()

plt.show()

> ### (10pts) Compare the final performance of your models on the test dataset
  * Examine the final performance of the baseline, trial1, trial2, and trial3 models.
  * Verify the performance is improved as you apply some regularization methods. If not, discuss why.
